<a href="https://colab.research.google.com/github/AnyaNiks/CatcherInTheRye_Python_Project/blob/main/CatherInTheRye_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PDF > TXT

In [3]:
!wget https://raw.githubusercontent.com/AnyaNiks/CatcherInTheRye_Python_Project/main/Rye.pdf

--2025-05-23 12:28:36--  https://raw.githubusercontent.com/AnyaNiks/CatcherInTheRye_Python_Project/main/Rye.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 555295 (542K) [application/octet-stream]
Saving to: ‘Rye.pdf’

Rye.pdf             100%[===================>] 542.28K  --.-KB/s    in 0.05s   

2025-05-23 12:28:36 (10.0 MB/s) - ‘Rye.pdf’ saved [555295/555295]



In [4]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 23.8 MB/s eta 0:00:00


In [5]:
import fitz

In [6]:
pdf = fitz.open("Rye.pdf")  # Open the PDF file

text = ""    # Initialize an empty string to store the extracted text
# Loop through each page of the PDF
for page in pdf:
    # Extract text from the page and add it to our variable
    text += page.get_text()

# Save the full extracted text to a .txt file
with open("Rye.txt", "w", encoding="utf-8") as f:
    f.write(text)

print("Done!")

Done!


In [7]:
from google.colab import files
files.download("Rye.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
import string

with open("Rye.txt", "r", encoding="utf-8") as f:
    text = f.read()

text = text.lower()
text = text.replace("“", '"').replace("”", '"').replace("’", "'")

for p in string.punctuation:
    text = text.replace(p, " ")

words = text.split()
print(words[:30])

['the', 'catcher', 'in', 'the', 'rye', 'by', 'j', 'd', 'salinger', 'to', 'my', 'mother', '1', 'if', 'you', 'really', 'want', 'to', 'hear', 'about', 'it', 'the', 'first', 'thing', 'you', 'll', 'probably', 'want', 'to', 'know']


In [9]:
import re  # We need the 're' module for regular expressions (used to remove extra spaces)

# Split the text into lines at each newline character ('\n')
lines = text.split("\n")

# Remove empty lines and strip leading/trailing spaces (this also removes lines that are just whitespace)
cleaned_lines = [line.strip() for line in lines if line.strip() != ""]

# Replace multiple spaces inside each line with a single space without touching quotes
cleaned_lines = [re.sub(r'(?<=\S) {2,}(?=\S)', ' ', line) for line in cleaned_lines]

# Join the cleaned lines back into one block of text
# Use two newlines ('\n\n') to separate paragraphs
cleaned_text = "\n\n".join(cleaned_lines)

# Save the cleaned text into a new file
with open("Rye_clean.txt", "w", encoding="utf-8") as f:
    f.write(cleaned_text)

# Step 6: Confirm that the cleaning is done
print(" cleaning done and saved to rye_clean.txt")

 cleaning done and saved to rye_clean.txt


In [10]:
from google.colab import files
files.download("Rye_clean.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>